In [1]:
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
FACEMESH_POINTS = mp_face_mesh.FACEMESH_TESSELATION  # all triangulated connections

In [15]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import scipy.ndimage

# -------------------------------------------------
# Initialize MediaPipe
# -------------------------------------------------
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

HAIR_MODEL_PATH = '/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/hair_segmenter.tflite'

# -------------------------------------------------
# Simplified Facial Feature Dictionaries
# -------------------------------------------------
FACIAL_FEATURES = {
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282, 295, 285, 336],
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],
    "nose_left": [122, 196, 236, 198, 209, 49, 48, 64, 98],
    "nose_right": [420, 360, 278, 294, 327, 326],
    "nostrils_left": [98, 240, 75, 60, 99, 97],
    "nostrils_right": [370, 354, 458, 290],
    "nose_center": [238, 241, 242, 370, 354, 94],
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],
    "left_eye_center": [468],
    "right_eye_center": [473],
}

EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# -------------------------------------------------
# Drawing helpers
# -------------------------------------------------
def smooth_points(points, sigma=5):
    points = np.array(points, dtype=np.float32)
    x_smooth = scipy.ndimage.gaussian_filter1d(points[:, 0], sigma=sigma)
    y_smooth = scipy.ndimage.gaussian_filter1d(points[:, 1], sigma=sigma)
    return list(zip(x_smooth.astype(int), y_smooth.astype(int)))

def draw_feature(image, landmarks, indices, color=(255, 255, 255), thickness=2, smooth=True):
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in indices]
    if smooth and len(points) >= 3:
        points = smooth_points(points, sigma=0.1)
    for i in range(len(points) - 1):
        cv2.line(image, points[i], points[i + 1], color, thickness)

def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_index, bottom_eyelid_index, color=(255, 255, 255)):
    H, W = image.shape[:2]
    pts = np.array([[landmarks[i].x * W, landmarks[i].y * H] for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))
    upper_y = landmarks[top_eyelid_index].y * H
    bottom_y = landmarks[bottom_eyelid_index].y * H
    num_points = 200
    for theta in np.linspace(0, 2*np.pi, num_points):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if upper_y <= y <= bottom_y:
            cv2.circle(image, (x, y), 1, color, 1)
    pupil_radius = int(radius * 0.3)
    for y in range(int(center[1]-pupil_radius), int(center[1]+pupil_radius)):
        for x in range(int(center[0]-pupil_radius), int(center[0]+pupil_radius)):
            if (x-center[0])**2 + (y-center[1])**2 <= pupil_radius**2 and upper_y <= y <= bottom_y:
                image[int(y), int(x)] = color

# -------------------------------------------------
# Main selective line drawing
# -------------------------------------------------
def image_to_minimal_line_drawing_simplified(image_path, output_path):
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255

    # --- FaceMesh landmarks ---
    with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5) as face_mesh:
        results_face = face_mesh.process(img_rgb)
        if results_face.multi_face_landmarks:
            face_landmarks = results_face.multi_face_landmarks[0]
            for name, indices in FACIAL_FEATURES.items():
                if "center" not in name:
                    draw_feature(final_line_drawing, face_landmarks.landmark, indices, color=0, thickness=2)

            # Eyeballs
            for iris_name, iris_indices in EYEBALLS.items():
                if "left" in iris_name:
                    draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices, 160, 145, color=0)
                else:
                    draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices, 385, 374, color=0)

    # --- Hair/body outline ---
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_seg = segmentor.process(img_rgb)
        if results_seg.segmentation_mask is not None:
            person_mask = (results_seg.segmentation_mask > 0.5).astype(np.uint8) * 255
            blurred = cv2.GaussianBlur(person_mask, (61, 61), 0)
            edges = cv2.Canny(blurred, 10, 50)
            edges = cv2.dilate(edges, np.ones((5, 5), np.uint8), 1)
            inverted = cv2.bitwise_not(edges)
            final_line_drawing = np.minimum(final_line_drawing, inverted)

    # --- Optional dedicated hair model ---
    if HAIR_MODEL_PATH:
        try:
            base_options = python.BaseOptions(model_asset_path=HAIR_MODEL_PATH)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
            options = vision.ImageSegmenterOptions(
                base_options=base_options,
                running_mode=vision.RunningMode.IMAGE,
                output_category_mask=True
            )
            with vision.ImageSegmenter.create_from_options(options) as segmenter:
                segmentation_result = segmenter.segment(mp_image)
                category_mask = segmentation_result.category_mask.numpy_view()
                hair_mask = np.where(category_mask == 1, 255, 0).astype(np.uint8)
                hair_mask = cv2.resize(hair_mask, (W, H), interpolation=cv2.INTER_NEAREST)
                blurred_hair = cv2.GaussianBlur(hair_mask, (41, 41), 0)
                edges = cv2.Canny(blurred_hair, 10, 50)
                edges = cv2.dilate(edges, np.ones((5, 5), np.uint8), 1)
                inverted = cv2.bitwise_not(edges)
                final_line_drawing = np.minimum(final_line_drawing, inverted)
        except Exception as e:
            print(f"Hair model error: {e}")

    # ---------------------------
    # Continuous single-line simplification
    # ---------------------------
    binary = cv2.threshold(final_line_drawing, 127, 255, cv2.THRESH_BINARY_INV)[1]

    # Merge nearby features
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    merged = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=3)
    merged = cv2.morphologyEx(merged, cv2.MORPH_OPEN, kernel, iterations=1)

    # Find all contours
    contours, _ = cv2.findContours(merged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if not contours:
        print("No contours found.")
        cv2.imwrite(output_path, final_line_drawing)
        return

    # Pick the largest contour (main face outline)
    contour = max(contours, key=cv2.contourArea)

    # Optional: merge all contours into one (if you want a full 'one-line' composite)
    # contour = np.vstack(contours)

    # Combine all contours to preserve internal details
    contour = np.vstack(contours)

    # Resample adaptively
    num_points = max(4000, len(contour) // 3)
    contour = contour[np.linspace(0, len(contour)-1, num=num_points, dtype=int)]

    # Fine simplification
    epsilon = 0.0008 * cv2.arcLength(contour.reshape(-1,1,2), True)
    contour = cv2.approxPolyDP(contour.reshape(-1,1,2), epsilon, True)

    # Light smoothing
    from scipy.ndimage import gaussian_filter1d
    sigma = 0.8
    contour = contour.squeeze()
    contour[:,0] = gaussian_filter1d(contour[:,0].astype(float), sigma=sigma)
    contour[:,1] = gaussian_filter1d(contour[:,1].astype(float), sigma=sigma)
    contour = contour.astype(int).reshape(-1,1,2)

    # Draw the smooth single line and save output
    single_line = np.ones_like(final_line_drawing) * 255
    cv2.polylines(single_line, [contour], isClosed=False, color=0, thickness=2)

    cv2.imwrite(output_path, single_line)
    print(f"✅ Saved continuous single-line drawing to: {output_path}")
# Example usage
if __name__ == "__main__":
    image_to_minimal_line_drawing_simplified('divija2.png', 'divija_line.png')


Processing: divija2.png


I0000 00:00:1761940475.110831 59735087 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1761940475.112650 59761977 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761940475.117370 59761982 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1761940475.127174 59735087 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1761940475.129008 59761992 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1761940475.183036 59735087 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1761940475.186205 59762003 inference_feedback_manager.cc:114] Feedback manager requires 

✅ Saved continuous single-line drawing to: divija_line.png


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize FaceMesh model (refine_landmarks=True gives iris points)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

# Load image
image = cv2.imread('divija2.png')
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process image to get landmarks
results = face_mesh.process(rgb_image)

# ---------------------------
# FACIAL FEATURE DICTIONARIES
# ---------------------------

FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282],
    "top_eye" : [414, 286, 258, 257, 259, 260, 467],  # upper eye contour
    "bottom_eye" : [453, 452, 451, 450, 449, 448],  # lower eye contour
    "top_eye_left": [190, 56, 28, 27, 29, 30, 247],
    "bottom_eye_left": [228, 229, 230, 231, 232, 233],
    # # Nose bridge and tip
    # "nose": [1, 2, 98, 327, 168, 197, 195, 5, 4, 45],
    # #"nose_bridge": [6, 197, 195, 5, 4],
    # "nose_tip": [1, 2, 98, 327],
    # Nose
    # "nose_bridge": [6, 197, 195, 5, 4],
    # "nose_outline_left": [98, 97, 2, 326],    # outer curve from bridge to nostril
    # "nose_outline_right": [327, 326, 2, 97],  # mirror outline
    # "nostril_left": [49, 50, 101, 118, 117],
    # "nostril_right": [279, 278, 330, 347, 348],
    # "nose_bottom": [97, 2, 326],  # base of nose

    # Lips
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],

    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],

    # Upper lip ridge (natural lip contour)
    "upper_lip": [61, 40, 37, 0, 267, 270, 409, 291],

    # Optional lower lip (for extra definition)
    "lower_lip": [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291],

    # Chin outline (jawline)
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],

    # # Ears
    # "left_ear": [234, 93, 132, 58, 172, 136],
    # "right_ear": [454, 323, 361, 288, 397, 365],

    # # Line between nose and lips (philtrum)
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],
    

    # Mid-eye centers (for eyeball placement)
    "left_eye_center": [468],  # available if using iris model
    "right_eye_center": [473],
}

# Iris (eyeball) indices (only exist when refine_landmarks=True)
EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# ---------------------------
# DRAWING HELPERS
# ---------------------------

def draw_feature(image, landmarks, indices, color=(255, 255, 255), thickness=1):
    """Draw line features (eyes, lips, etc.)"""
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in indices]
    for i in range(len(points) - 1):
        cv2.line(image, points[i], points[i + 1], color, thickness)


def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_indices, bottom_eyelid_indices, color=(255, 255, 255)):
    """
    Draw eyeball circle clipped at eyelids.
    - iris_indices: points of iris
    - top_eyelid_indices: upper eyelid contour
    - bottom_eyelid_indices: lower eyelid contour
    """
    # Iris points
    pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                    for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))

    # Eyelid bounds
    top_pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                        for i in top_eyelid_indices])
    bottom_pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                           for i in bottom_eyelid_indices])
    y_top = np.min(top_pts[:, 1])
    y_bottom = np.max(bottom_pts[:, 1])

    # Draw circle as points only within eyelid bounds
    num_points = 100
    for theta in np.linspace(0, 2 * np.pi, num_points):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if y_top <= y <= y_bottom:
            cv2.circle(image, (x, y), 1, color, 1)  # small point to draw outline

    # Draw pupil at iris center
    pupil_radius = int(radius * 0.3)
    pupil_top = max(int(center[1] - pupil_radius), int(y_top))
    pupil_bottom = min(int(center[1] + pupil_radius), int(y_bottom))
    for y in range(pupil_top, pupil_bottom):
        for x in range(int(center[0] - pupil_radius), int(center[0] + pupil_radius)):
            if (x - center[0])**2 + (y - center[1])**2 <= pupil_radius**2:
                image[y, x] = color


# ---------------------------
# MAIN LOGIC
# ---------------------------

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        # Draw facial line features
        for name, indices in FACIAL_FEATURES.items():
            draw_feature(image, face_landmarks.landmark, indices)

        for iris_name, iris_indices in EYEBALLS.items():
            if "left" in iris_name:
                draw_eyeball_clipped(image, face_landmarks.landmark, iris_indices,
                             FACIAL_FEATURES["top_eye_left"], FACIAL_FEATURES["bottom_eye_left"])
            else:
                draw_eyeball_clipped(image, face_landmarks.landmark, iris_indices,
                             FACIAL_FEATURES["top_eye"], FACIAL_FEATURES["bottom_eye"])

# Show output
cv2.imwrite("face.png", image)


I0000 00:00:1760856941.733401 48421607 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760856941.734935 48456174 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760856941.739918 48456180 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


True

In [13]:
# very very good code
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import scipy.ndimage

# Initialize MediaPipe
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

HAIR_MODEL_PATH = '/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/hair_segmenter.tflite'

# ---------------------------
# Simplified Facial Feature Dictionaries
# ---------------------------

FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282, 295, 285, 336],

    # Upper / lower eyelid contours for clipped eyeballs
    "top_eye": [414, 286, 258, 257, 259, 260, 467],
    "bottom_eye": [453, 452, 451, 450, 449, 448],
    "top_eye_left": [190, 56, 28, 27, 29, 30, 247],
    "bottom_eye_left": [228, 229, 230, 231, 232, 233],

    # Lips
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],
    "upper_lip": [61, 40, 37, 0, 267, 270, 409, 291],
    "lower_lip": [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291],

    # Chin / jawline
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],
    # actual nose:
    "nose_left": [122, 196, 236, 198, 209, 49, 48, 64, 98],
    "nose_right": [420, 360, 278, 294, 327, 326],
    "nostrils_left" : [98, 240, 75, 60, 99, 97],
    "nostrils_right" : [370, 354, 458, 290],
    "nose_center": [238, 241,242,370, 354, 94],
    # Nose to lips (philtrum)
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],

    # Mid-eye centers for eyeball placement
    "left_eye_center": [468],
    "right_eye_center": [473],
}   

# Iris (eyeball) indices
EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# ---------------------------
# Drawing helpers
# ---------------------------

def smooth_points(points, sigma=5):
    """
    Smooth a list of points using Gaussian smoothing independently on x and y.
    points: list of (x, y)
    sigma: Gaussian kernel standard deviation
    """
    
    points = np.array(points, dtype=np.float32)
    x_smooth = scipy.ndimage.gaussian_filter1d(points[:, 0], sigma=sigma)
    y_smooth = scipy.ndimage.gaussian_filter1d(points[:, 1], sigma=sigma)
    return list(zip(x_smooth.astype(int), y_smooth.astype(int)))
def draw_feature(image, landmarks, indices, color=(255, 255, 255), thickness=2, smooth=True):
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in indices]
    if smooth and len(points) >= 3:
        points = smooth_points(points, sigma=0.1)
    for i in range(len(points) - 1):
        cv2.line(image, points[i], points[i + 1], color, thickness)

def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_index, bottom_eyelid_index, color=(255, 255, 255)):
    """
    Draw an eyeball circle clipped at the top eyelid landmark.
    - iris_indices: list of iris landmarks
    - top_eyelid_index: single landmark index for upper eyelid to clip at
    - bottom_eyelid_indices: list of lower eyelid landmarks
    """
    H, W = image.shape[:2]
    
    # Iris points
    pts = np.array([[landmarks[i].x * W, landmarks[i].y * H] for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))
    
    # Upper eyelid landmark coordinate
    upper_y = landmarks[top_eyelid_index].y * H

    # Bottom eyelid bounds
    bottom_y = np.array([landmarks[bottom_eyelid_index].y * H])
    y_bottom = np.max(bottom_y)
    
    # Draw circle outline, skip points above upper eyelid landmark
    num_points = 200
    for theta in np.linspace(0, 2*np.pi, num_points):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if upper_y <= y <= y_bottom:
            cv2.circle(image, (x, y), 1, color, 1)
    
    # Draw pupil clipped at the same bounds
    pupil_radius = int(radius * 0.3)
    pupil_top = max(int(center[1] - pupil_radius), int(upper_y))
    pupil_bottom = min(int(center[1] + pupil_radius), int(y_bottom))
    
    for y in range(pupil_top, pupil_bottom):
        for x in range(int(center[0] - pupil_radius), int(center[0] + pupil_radius)):
            if (x - center[0])**2 + (y - center[1])**2 <= pupil_radius**2:
                image[y, x] = color
# ---------------------------
# Main selective line drawing
# ---------------------------

def image_to_minimal_line_drawing_simplified(image_path, output_path):
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255

    # FaceMesh
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        results_face = face_mesh.process(img_rgb)
        if results_face.multi_face_landmarks:
            face_landmarks = results_face.multi_face_landmarks[0]

    # Draw simplified facial features on final_line_drawing
    for name, indices in FACIAL_FEATURES.items():
        if "left_eye_center" in name or "right_eye_center" in name:
            continue
        draw_feature(final_line_drawing, face_landmarks.landmark, indices, color=0, thickness=2)

    # Draw clipped eyeballs on final_line_drawing
    for iris_name, iris_indices in EYEBALLS.items():
        if "left" in iris_name:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices,
                                160, 145, color=0)
        else:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices,
                                385, 374, color=0)

    # Selfie Segmentation for hair/body outline
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        if results_segmentation.segmentation_mask is not None:
            person_mask = (results_segmentation.segmentation_mask > 0.5).astype(np.uint8) * 255
            blurred_outline = cv2.GaussianBlur(person_mask, (61,61), 0)
            edges = cv2.Canny(blurred_outline, 10, 50)
            edges = cv2.dilate(edges, np.ones((5,5),np.uint8), iterations=1)
            inverted = cv2.bitwise_not(edges)
            final_line_drawing = np.minimum(final_line_drawing, inverted)

    # Optional dedicated hair model
    if HAIR_MODEL_PATH:
        try:
            base_options = python.BaseOptions(model_asset_path=HAIR_MODEL_PATH, delegate=None)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
            options = vision.ImageSegmenterOptions(
                base_options=base_options,
                running_mode=vision.RunningMode.IMAGE,
                output_category_mask=True
            )
            with vision.ImageSegmenter.create_from_options(options) as segmenter:
                segmentation_result = segmenter.segment(mp_image)
                category_mask = segmentation_result.category_mask.numpy_view()
                hair_mask = np.where(category_mask == 1, 255, 0).astype(np.uint8)
                hair_mask = cv2.resize(hair_mask, (W,H), interpolation=cv2.INTER_NEAREST)
                blurred_hair = cv2.GaussianBlur(hair_mask, (41,41),0)
                edges = cv2.Canny(blurred_hair,10,50)
                edges = cv2.dilate(edges,np.ones((5,5),np.uint8),iterations=1)
                inverted = cv2.bitwise_not(edges)
                final_line_drawing = np.minimum(final_line_drawing, inverted)
        except Exception as e:
            print(f"Hair model error: {e}")
        
         # Save output
        # ---------------------------
    # Line simplification step
    # ---------------------------
    # Convert to binary (0/255)
    binary = cv2.threshold(final_line_drawing, 127, 255, cv2.THRESH_BINARY_INV)[1]

    # Morphological closing to merge nearby lines
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    merged = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Distance-based thinning (skeletonization)
    dist = cv2.distanceTransform(merged, cv2.DIST_L2, 5)
    skeleton = np.zeros_like(merged)
    skeleton[dist > 1] = 255   # threshold controls how aggressively to keep single lines

    # # Optional contour simplification (reduces jittery edges)
    # contours, _ = cv2.findContours(skeleton, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # simplified = np.ones_like(final_line_drawing) * 255
    # for cnt in contours:
    #     epsilon = 0.001 * cv2.arcLength(cnt, True)
    #     approx = cv2.approxPolyDP(cnt, epsilon, True)
    #     cv2.drawContours(simplified, [approx], -1, 0, 1)

    # final_line_drawing = simplified

    cv2.imwrite(output_path, final_line_drawing)
    print(f"Saved simplified line drawing to: {output_path}")

# Example usage
image_to_minimal_line_drawing_simplified('niru.png', 'niru_line.png')


Processing: niru.png
Saved simplified line drawing to: niru_line.png


I0000 00:00:1762136791.370415 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762136791.371117 61442778 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762136791.374714 61442779 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1762136791.382858 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762136791.383670 61442790 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1762136791.400570 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762136791.401784 61442800 inference_feedback_manager.cc:114] Feedback manager requires 

In [9]:
import csv
import cv2
import numpy as np

def csv_to_video(
    csv_path,
    out_video="csv_draw_anim.mp4",
    frame_rate=60,
    canvas_size=(512,512),
    line_thickness=1,
    speed=1,
):
    """
    Visualize draw_coords.csv as a video showing pen-down and pen-up moves.
    - speed: how many CSV points per frame (higher = faster animation)
    """

    # --- Load coordinates ---
    coords = []
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            coords.append((int(row["x"]), int(row["y"]), int(row["pen"])))

    if not coords:
        raise ValueError("CSV is empty or invalid")

    # --- Normalize / scale coordinates to fit canvas ---
    xs, ys = [p[0] for p in coords], [p[1] for p in coords]
    minx, maxx, miny, maxy = min(xs), max(xs), min(ys), max(ys)
    W, H = canvas_size
    scale = min(W / (maxx - minx + 1e-5), H / (maxy - miny + 1e-5))
    offset_x = -minx
    offset_y = -miny

    def transform(pt):
        x = int((pt[0] + offset_x) * scale)
        y = int((pt[1] + offset_y) * scale)
        return x, y

    # --- Setup video ---
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    vid = cv2.VideoWriter(out_video, fourcc, frame_rate, (W, H))
    canvas = np.ones((H, W, 3), np.uint8) * 255
    pen_color = (0, 0, 0)
    travel_color = (180, 180, 180)
    pen_tip_color = (0, 0, 255)

    # --- Draw progressively ---
    prev_pt = None
    for i in range(0, len(coords), speed):
        x, y, pen = coords[i]
        pt = transform((x, y))

        # Draw
        if pen == 1 and prev_pt is not None:
            cv2.line(canvas, prev_pt, pt, pen_color, line_thickness)
        elif pen == 0 and prev_pt is not None:
            cv2.line(canvas, prev_pt, pt, travel_color, 1)

        # Draw pen tip
        frame = canvas.copy()
        cv2.circle(frame, pt, 2, pen_tip_color, -1)
        vid.write(frame)

        prev_pt = pt

    vid.release()
    print(f"✅ Saved drawing animation: {out_video}")

# Example usage
if __name__ == "__main__":
    csv_to_video("draw_coords.csv", "csv_draw_anim.mp4", speed=3, canvas_size=(800,800))


✅ Saved drawing animation: csv_draw_anim.mp4


In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

# -------------------------------------------------
# CONFIGURATION
# -------------------------------------------------
INPUT_PATH = "divija_line.png"
OUTPUT_CSV = "drawing_path.csv"
OUTPUT_VIS = "extracted_paths.png"

# parameters
THRESH = 200
EPSILON = 1.5
DOWNSAMPLE_STEP = 3

# -------------------------------------------------
# 1. Load and preprocess
# -------------------------------------------------
img = cv2.imread(INPUT_PATH, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not read image at {INPUT_PATH}")

_, binary = cv2.threshold(img, THRESH, 255, cv2.THRESH_BINARY_INV)
binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

# thin lines to 1-pixel width
skeleton = cv2.ximgproc.thinning(binary)

# -------------------------------------------------
# 2. Extract contours (continuous strokes)
# -------------------------------------------------
contours, _ = cv2.findContours(skeleton, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print(f"Found {len(contours)} raw contours")

paths = []
for cnt in contours:
    approx = cv2.approxPolyDP(cnt, EPSILON, False)
    pts = approx[:, 0, :]
    if len(pts) > DOWNSAMPLE_STEP:
        pts = pts[::DOWNSAMPLE_STEP]
    if len(pts) > 2:
        paths.append(pts)

print(f"Retained {len(paths)} simplified paths")

# -------------------------------------------------
# 3. Print and save extracted paths
# -------------------------------------------------
for i, path in enumerate(paths):
    print(f"Path {i}: {len(path)} points")

# Save combined coordinates for downstream use
all_coords = np.concatenate(paths, axis=0)
np.savetxt(OUTPUT_CSV, all_coords, fmt="%.1f", delimiter=",", header="x,y")
print(f"Saved coordinates to {OUTPUT_CSV}")

# -------------------------------------------------
# 4. Visualize on blank canvas
# -------------------------------------------------
# Create a white canvas same size as input
canvas = np.ones_like(img) * 255
canvas_color = cv2.cvtColor(canvas, cv2.COLOR_GRAY2BGR)

color_map = plt.cm.jet(np.linspace(0, 1, len(paths)))

for i, pts in enumerate(paths):
    color = tuple(int(c * 255) for c in color_map[i][:3])
    for j in range(len(pts) - 1):
        cv2.line(canvas_color, tuple(pts[j]), tuple(pts[j+1]), color, 1)

cv2.imwrite(OUTPUT_VIS, canvas_color)
print(f"Saved extracted path visualization as {OUTPUT_VIS}")


Found 245 raw contours
Retained 91 simplified paths
Path 0: 6 points
Path 1: 3 points
Path 2: 4 points
Path 3: 3 points
Path 4: 4 points
Path 5: 5 points
Path 6: 3 points
Path 7: 3 points
Path 8: 5 points
Path 9: 3 points
Path 10: 10 points
Path 11: 4 points
Path 12: 3 points
Path 13: 3 points
Path 14: 18 points
Path 15: 5 points
Path 16: 3 points
Path 17: 3 points
Path 18: 3 points
Path 19: 3 points
Path 20: 3 points
Path 21: 14 points
Path 22: 3 points
Path 23: 3 points
Path 24: 3 points
Path 25: 3 points
Path 26: 3 points
Path 27: 3 points
Path 28: 3 points
Path 29: 3 points
Path 30: 3 points
Path 31: 3 points
Path 32: 3 points
Path 33: 3 points
Path 34: 3 points
Path 35: 3 points
Path 36: 3 points
Path 37: 3 points
Path 38: 3 points
Path 39: 3 points
Path 40: 3 points
Path 41: 3 points
Path 42: 3 points
Path 43: 3 points
Path 44: 3 points
Path 45: 3 points
Path 46: 3 points
Path 47: 3 points
Path 48: 3 points
Path 49: 3 points
Path 50: 3 points
Path 51: 3 points
Path 52: 3 points
P

In [28]:
# very very very good code oolala so sexy
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import scipy.ndimage

# -------------------------------------------------
# Initialize MediaPipe
# -------------------------------------------------
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

HAIR_MODEL_PATH = '/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/hair_segmenter.tflite'

# -------------------------------------------------
# Facial feature landmark groups
# -------------------------------------------------
FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282, 295, 285, 336],

    # Upper / lower eyelid contours for clipped eyeballs
    "top_eye": [414, 286, 258, 257, 259, 260, 467],
    "bottom_eye": [453, 452, 451, 450, 449, 448],
    "top_eye_left": [190, 56, 28, 27, 29, 30, 247],
    "bottom_eye_left": [228, 229, 230, 231, 232, 233],

    # Lips
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],
    "upper_lip": [61, 40, 37, 0, 267, 270, 409, 291],
    "lower_lip": [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291],

    # Chin / jawline
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],
    # actual nose:
    "nose_left": [122, 196, 236, 198, 209, 49, 48, 64, 98],
    "nose_right": [420, 360, 278, 294, 327, 326],
    "nostrils_left" : [98, 240, 75, 60, 99, 97],
    "nostrils_right" : [370, 354, 458, 290],
    "nose_center": [238, 241,242,370, 354, 94],
    # Nose to lips (philtrum)
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],

    # Mid-eye centers for eyeball placement
    "left_eye_center": [468],
    "right_eye_center": [473],
}   


EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# -------------------------------------------------
# Helpers
# -------------------------------------------------
def draw_feature(image, landmarks, indices, color=0, thickness=1):
    pts = [(int(landmarks[i].x * image.shape[1]),
            int(landmarks[i].y * image.shape[0])) for i in indices]
    for i in range(len(pts) - 1):
        cv2.line(image, pts[i], pts[i + 1], color, thickness)

def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_index, bottom_eyelid_index, color=0):
    H, W = image.shape[:2]
    pts = np.array([[landmarks[i].x * W, landmarks[i].y * H] for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))
    upper_y = landmarks[top_eyelid_index].y * H
    bottom_y = landmarks[bottom_eyelid_index].y * H
    for theta in np.linspace(0, 2*np.pi, 200):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if upper_y <= y <= bottom_y and 0 <= y < H and 0 <= x < W:
            image[y, x] = color

# -------------------------------------------------
# Main
# -------------------------------------------------
def image_to_minimal_line_drawing_simplified(image_path, output_path):
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255

    # ---- FaceMesh ----
    with mp_face_mesh.FaceMesh(
        static_image_mode=True, max_num_faces=1,
        refine_landmarks=True, min_detection_confidence=0.5
    ) as face_mesh:
        res = face_mesh.process(img_rgb)
        if not res.multi_face_landmarks:
            print("No face found."); return
        face_landmarks = res.multi_face_landmarks[0]

    # ---- Facial features ----
    for name, idxs in FACIAL_FEATURES.items():
        if "center" in name: continue
        draw_feature(final_line_drawing, face_landmarks.landmark, idxs)
    for iris_name, iris_indices in EYEBALLS.items():
        if "left" in iris_name:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices, 160, 145)
        else:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices, 385, 374)

    # ---- Hair segmentation ----
    hair_outline = np.ones((H, W), dtype=np.uint8) * 255
    hair_mask = None
    try:
        base_opts = python.BaseOptions(model_asset_path=HAIR_MODEL_PATH)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
        opts = vision.ImageSegmenterOptions(
            base_options=base_opts,
            running_mode=vision.RunningMode.IMAGE,
            output_category_mask=True
        )
        with vision.ImageSegmenter.create_from_options(opts) as segmenter:
            seg = segmenter.segment(mp_image)
            cat_mask = seg.category_mask.numpy_view()
            hair_mask = np.where(cat_mask == 1, 255, 0).astype(np.uint8)
            hair_mask = cv2.resize(hair_mask, (W, H), interpolation=cv2.INTER_NEAREST)
            cs, _ = cv2.findContours(hair_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            for c in cs:
                eps = 0.002 * cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps, True)
                cv2.drawContours(hair_outline, [approx], -1, 0, 1)
            hair_outline = cv2.ximgproc.thinning(255 - hair_outline)
            hair_outline = 255 - hair_outline
            final_line_drawing = np.minimum(final_line_drawing, hair_outline)
    except Exception as e:
        print(f"Hair model error: {e}")

    # ---- Selfie segmentation ----
    outline = np.ones((H, W), dtype=np.uint8) * 255
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as seg:
        res = seg.process(img_rgb)
        if res.segmentation_mask is not None:
            body_mask = (res.segmentation_mask > 0.5).astype(np.uint8) * 255
            cs, _ = cv2.findContours(body_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            for c in cs:
                eps = 0.0015 * cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, eps, True)
                cv2.drawContours(outline, [approx], -1, 0, 1)

    # ---- Hair-priority merge ----
    if hair_mask is not None:
        hair_bin = cv2.threshold(hair_outline, 127, 255, cv2.THRESH_BINARY_INV)[1]
        body_bin = cv2.threshold(outline, 127, 255, cv2.THRESH_BINARY_INV)[1]
        dist_to_hair = cv2.distanceTransform(255 - hair_bin, cv2.DIST_L2, 5)
        proximity_thresh = 50  # px; tune this
        body_bin[dist_to_hair < proximity_thresh] = 0
        outline = 255 - body_bin

    # ---- Combine & finalize ----
    final_line_drawing = np.minimum(final_line_drawing, outline)
    binary = cv2.threshold(final_line_drawing, 127, 255, cv2.THRESH_BINARY_INV)[1]
    skeleton = cv2.ximgproc.thinning(binary)
    final_line_drawing = 255 - skeleton

    cv2.imwrite(output_path, final_line_drawing)
    print(f"✅ Saved single-pixel line drawing to: {output_path}")

# -------------------------------------------------
# Example usage
# -------------------------------------------------
image_to_minimal_line_drawing_simplified('woman.jpeg', 'woman_line.png')


Processing: woman.jpeg
✅ Saved single-pixel line drawing to: woman_line.png


I0000 00:00:1762137891.345272 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762137891.346666 61472432 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762137891.350605 61472430 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1762137891.361280 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762137891.362746 61472443 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1762137891.385742 61411922 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1762137891.387352 61472462 inference_feedback_manager.cc:114] Feedback manager requires 

In [ ]:

# -------------------------------------------------
# Camera + face capture parameters
# -------------------------------------------------
CAPTURE_PATH = "captured_face.png"
CENTER_TOLERANCE = 0.1  # 10% of frame width/height
FACE_DETECT_CONF = 0.6

mp_face = mp.solutions.face_detection

def capture_centered_face(output_path=CAPTURE_PATH):
    cap = cv2.VideoCapture(0)
    detector = mp_face.FaceDetection(model_selection=0, min_detection_confidence=FACE_DETECT_CONF)
    print("📷 Starting camera. Center your face to capture automatically...")

    captured = False
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Camera read failed.")
            break

        H, W, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = detector.process(rgb)

        if results.detections:
            for det in results.detections:
                bbox = det.location_data.relative_bounding_box
                cx = bbox.xmin + bbox.width / 2
                cy = bbox.ymin + bbox.height / 2
                dx = abs(cx - 0.5)
                dy = abs(cy - 0.5)

                # Visual guide
                cv2.rectangle(frame, 
                              (int((0.5 - CENTER_TOLERANCE)*W), int((0.5 - CENTER_TOLERANCE)*H)),
                              (int((0.5 + CENTER_TOLERANCE)*W), int((0.5 + CENTER_TOLERANCE)*H)),
                              (0,255,0), 2)
                cv2.putText(frame, "Align your face in green box", (30,40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                if dx < CENTER_TOLERANCE and dy < CENTER_TOLERANCE:
                    cv2.putText(frame, "✅ Face centered - capturing...", (30,70),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                    cv2.imwrite(output_path, frame)
                    captured = True
                    break

        cv2.imshow("Align Your Face", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or captured:
            break

    cap.release()
    cv2.destroyAllWindows()
    detector.close()

    if captured:
        print(f"✅ Captured image saved to {output_path}")
        return output_path
    else:
        print("❌ No capture made.")
        return None

# -------------------------------------------------
# Main combined flow
# -------------------------------------------------
def capture_and_generate():
    captured_path = capture_centered_face()
    if captured_path and os.path.exists(captured_path):
        # Run the full pipeline on captured image
        line_path = "captured_line.png"
        csv_path = "captured_coords.csv"
        video_path = "captured_draw.mp4"
        image_to_minimal_line_drawing_simplified(captured_path, line_path, csv_path, video_path)

# -------------------------------------------------
# Run
# -------------------------------------------------
if __name__ == "__main__":
    capture_and_generate()
